In [7]:
# Imports
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np
from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import bokeh.io
import bokeh.plotting

## CRN for Produce 2N w/ Broccoli Aptamer ##

In [8]:
# Create CRN of Produce 2N
Core2_off = Species("Core2")
#rR1 = Species("rR1")
#Broccoli_Aptamer = Species("BrocApt")
#dA1 = Species("dA1" )
DFHBI = Species("DFHBI")

Core2 = Genelet(Core2_off, transcript= Broccoli_Aptamer , activator= dA1  , inhibitor= rR1)
M_Prod2 = Mixture(name = "Produce2_test", components = [Core2], parameter_file = "default_parameters.txt")
repr(M_Prod2)
CRN_Prod2 = M_Prod2.compile_crn()

#rxn1 = ([Broccoli_Aptamer, DFHBI_dye], [ComplexSpecies([Broccoli_Aptamer, DFHBI_dye], name = "Broc_flu"), k = 9.96e-2)
rxn1 = Reaction([Broccoli_Aptamer,DFHBI], [ComplexSpecies([Broccoli_Aptamer, DFHBI], name = "Flu1")], k = 9.96e-2)
CRN_Prod2.add_reactions([rxn1])

print(CRN_Prod2.pretty_print())

Species (11) = {0. Core2_OFF, 1. complex[Core2_OFF:dA1], 2. BrocApt, 3. dA1, 4. rR1, 5. protein[RNAP], 6. protein[RNAseH], 7. complex[rR1:dA1], 8. complex[complex[Core2_OFF:dA1]:protein[RNAP]], 9. complex[Core2_OFF:protein[RNAP]], 10. complex[complex[rR1:dA1]:protein[RNAseH]]}
Reactions (10) = [
0. Core2_OFF + dA1 --> complex[Core2_OFF:dA1]        
        massaction: k_f(Core2_OFF,dA1)=0.00394*Core2_OFF*dA1
1. complex[Core2_OFF:dA1] + rR1 --> Core2_OFF + complex[rR1:dA1]        
        massaction: k_f(complex[Core2_OFF:dA1],rR1)=0.0696*complex[Core2_OFF:dA1]*rR1
2. dA1 + rR1 --> complex[rR1:dA1]        
        massaction: k_f(dA1,rR1)=0.0696*dA1*rR1
3. complex[Core2_OFF:dA1] + protein[RNAP] <--> complex[complex[Core2_OFF:dA1]:protein[RNAP]]        
        massaction: k_f(complex[Core2_OFF:dA1],protein[RNAP])=38.857142857142854*complex[Core2_OFF:dA1]*protein[RNAP]
        k_r(complex[complex[Core2_OFF:dA1]:protein[RNAP]])=10.0*complex[complex[Core2_OFF:dA1]:protein[RNAP]]
4. complex

In [9]:
# BioSCRAPE simulation of Produce2N with No repressors

io = {"Core2_OFF": 5000, "dA1": 5000, "rR1": 0, "BrocApt": 0, "DFHBI": 60000, 
      "protein_RNAseH":0, "protein_RNAP":200}
# For label convenience
x = 'Time (seconds)'
y = 'Concentration (uM)'
timepoints = np.linspace(0, 5000, 1000)
R = CRN_Prod2.simulate_with_bioscrape(timepoints, initial_condition_dict = io)

bokeh.io.output_notebook()
p = bokeh.plotting.figure(
    plot_width = 300, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,
)
p.circle(timepoints, R["Core2_OFF"], legend_label = "OFF switch 2", color = "orange")
p.circle(timepoints, R["complex_Core2_ON"], legend_label = "ON switch 2" , color = "red")
p.legend.click_policy="hide"

r = bokeh.plotting.figure(
    plot_width = 300, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,)
r.circle(timepoints, R["complex_Flu1"], legend_label = "Fluoresence", color = "darkgreen")
r.legend.location = "bottom_right"

s = bokeh.plotting.figure(
    plot_width = 300, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,)
s.circle(timepoints, R["BrocApt"], legend_label = "Broc Apt", color = "green")
s.legend.click_policy="hide"
s.legend.location = "top_left"

#### Attempt to track DFHBI dye molecules ####
#### Ended with key error #####
t = bokeh.plotting.figure(
    plot_width = 300, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,)
t.circle(timepoints, R["DFHBI"], legend_label = "DFHBI molecules", color = "lightgreen")
t.legend.click_policy="hide"
t.legend.location = "top_right"

bokeh.io.show(row (p, s, r, t))
warnings.filterwarnings("ignore")

C:\Users\Jeremiah\anaconda3\lib\site-packages\html5lib\_trie\_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
C:\Users\Jeremiah\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Jeremiah\anaconda3\lib\site-packages\biocrnpyler-0.2.1-py3.7.egg\biocrnpyler\chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: DFHBI, complex_Flu1, 
C:\Users\Jeremiah\anaconda3\lib\site-packages\scipy\integrate\odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxste

Loading BokehJS ...

In [10]:
export_png(row (p, s, r, t), filename = "Produce2plots.png")

'C:\\Users\\Jeremiah\\OneDrive\\Documents\\GitHub\\jls_murray_2020SURF\\Produce2plots.png'